In [2]:
!pip install --upgrade --quiet opik langchain langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 580.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.5/185.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/3

In [3]:
import opik

opik.configure(use_local=False)

OPIK: Your Opik cloud API key is available at https://www.comet.com/api/my/settings/.


Please enter your Opik Cloud API key:··········
Do you want to use "sumitmishra5504" workspace? (Y/n)Y


OPIK: Saved configuration to a file: /root/.opik.config


In [4]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


## Model 1: gpt-4o-mini

In [5]:
from opik import Opik, track, DatasetItem
from opik.evaluation import evaluate
from opik.evaluation.metrics import Equals, Hallucination
from opik.integrations.openai import track_openai
import openai

# Define the task to evaluate
openai_client = track_openai(openai.OpenAI())
os.environ["OPIK_PROJECT_NAME"] = "opik-model-comparison"

MODEL = "gpt-4o-mini"

from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=MODEL,
    max_tokens=200
)

@track(name="gpt_4o_mini_tracing")
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below and just give sentiment as response.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>

    What is the sentiment of statement? Please respond with a only one word. e.g. sadness"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

# Define the evaluation task
def evaluation_task(x: DatasetItem):
    return {
        "input": x.input['input'],
        "output": get_sentiment(build_input_prompt(x.input['input'])),
        "expected_output": x.expected_output['golden_answer']
    }

# Create a simple dataset
client = Opik()
dataset = client.get_dataset(name="Sentiment Dataset")

# Define the metrics
from opik.evaluation.metrics import ContextPrecision, ContextRecall

metric1 = ContextPrecision()
metric2 = ContextRecall()

evaluation = evaluate(
    experiment_name="gpt-4o-mini comp",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[metric1, metric2],
    experiment_config={
        "model": MODEL
    }
)

<ipython-input-5-6fc8c785aace>:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
Evaluation: 100%|██████████| 20/20 [00:05<00:00,  3.72it/s]


╭─ Sentiment Dataset (20 samples) ───────╮
│                                        │
│ Total time:        00:00:05            │
│ Number of samples: 20                  │
│                                        │
│ context_precision_metric: 0.4200 (avg) │
│ context_recall_metric: 0.6650 (avg)    │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

## Model 2: claude-3-sonnet-20240229

In [6]:
ClaudeapiKey = getpass.getpass("Enter your Claude API key: ")

Enter your Claude API key: ··········


In [12]:
from opik import Opik, track, DatasetItem
from opik.evaluation import evaluate
from opik.evaluation.metrics import Equals, Hallucination
from opik.integrations.openai import track_openai
import openai
import urllib
import json

# Define the task to evaluate
openai_client = track_openai(openai.OpenAI())
os.environ["OPIK_PROJECT_NAME"] = "opik-model-comparison"

MODEL = "claude-3-sonnet-20240229"

def get_sentiment(messages):
    """Make an API call to the Anthropic Claude model."""
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        'x-api-key': ClaudeapiKey,
        'anthropic-version': '2023-06-01',
        'content-type': 'application/json'
    }
    data = json.dumps({
        "model": "claude-3-sonnet-20240229",
        "max_tokens": 200,
        "messages": messages
    }).encode('utf-8')

    req = urllib.request.Request(url, data=data, headers=headers, method='POST')

    try:
        with urllib.request.urlopen(req) as response:
            output = response.read()
            return json.loads(output)['content'][0]['text']
    except urllib.error.HTTPError as e:
        print(f"HTTP error: {e.code} - {e.reason}")
        print(f"Response body: {e.read().decode()}")

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below and just give sentiment as response.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>

    What is the sentiment of statement? Please respond with a only one word. e.g. sadness"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

@track(name="claude_3_sonnet_20240229_tracing")
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below and just give sentiment as response.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>

    What is the sentiment of statement? Please respond with a only one word. e.g. sadness"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

# Define the evaluation task
def evaluation_task(x: DatasetItem):
    return {
        "input": x.input['input'],
        "output": get_sentiment(build_input_prompt(x.input['input'])),
        "expected_output": x.expected_output['golden_answer']
    }

# Create a simple dataset
client = Opik()
dataset = client.get_dataset(name="Sentiment Dataset")

# Define the metrics
from opik.evaluation.metrics import ContextPrecision, ContextRecall

metric1 = ContextPrecision()
metric2 = ContextRecall()

evaluation = evaluate(
    experiment_name="claude-3-sonnet-20240229 comp",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[metric1, metric2],
    experiment_config={
        "model": MODEL
    }
)

Evaluation: 100%|██████████| 20/20 [00:06<00:00,  3.15it/s]


╭─ Sentiment Dataset (20 samples) ───────╮
│                                        │
│ Total time:        00:00:06            │
│ Number of samples: 20                  │
│                                        │
│ context_precision_metric: 0.4700 (avg) │
│ context_recall_metric: 0.6450 (avg)    │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...